# Import libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import sklearn
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import scipy.stats as stats
import math
from geopy.geocoders import Nominatim
import re
import pickle

# Train-Test
from sklearn.model_selection import train_test_split

# Encoder
from sklearn.preprocessing import OneHotEncoder

# Location clustering
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import haversine_distances

# Scaler
from sklearn.preprocessing import StandardScaler

# Imputation
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.impute import KNNImputer

# Normalization
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer

## Drop columns

In [2]:
def drop_cols(df,cols):
    df_ = df.copy()
    df_.drop(cols,axis=1,inplace=True)
    return df_

## Variable Encoding

In [3]:
# class GeneralEncoder():
#     def __init__(self):
#         # Wind direction (ordinal)
#         self.windDirDic = {
#             "E": 0,
#             "ENE": 1,
#             "NE": 2,
#             "NNE": 3,
#             "N": 4,
#             "NNW": 5,
#             "NW": 6,
#             "WNW": 7,
#             "W": 8,
#             "WSW": 9,
#             "SW": 10,
#             "SSW": 11,
#             "S": 12,
#             "SSE": 13,
#             "SE": 14,
#             "ESE": 15,
#         }
    
#     def fit(self):
#         pass

#     def transform(self, df):
#         df_ = df.copy(df)
        
#         df_.replace({"No":0,"Yes":1},inplace=True)
#         df_.replace(self.windDirDic,inplace=True)
#         return df_

# from sklearn import set_config
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer

# set_config(display='diagram')
# myPipeline = Pipeline(steps=[
#     ('general_encoding', GeneralEncoder()),
#     ])

In [4]:
def get_name_with_space(loc):
    return re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', loc)

def encode_variables(df):
    df_ = df.copy()

    # Binaries
    df_.replace({"No":0,"Yes":1},inplace=True)

    # Wind direction (ordinal)
    windDirDic = {
        "E": 0,
        "ENE": 1,
        "NE": 2,
        "NNE": 3,
        "N": 4,
        "NNW": 5,
        "NW": 6,
        "WNW": 7,
        "W": 8,
        "WSW": 9,
        "SW": 10,
        "SSW": 11,
        "S": 12,
        "SSE": 13,
        "SE": 14,
        "ESE": 15,
    }

    df_.replace(windDirDic,inplace=True)

    # Location
    # Get unique locations and get names with spaces
    locations = np.array(df_['Location'].unique(),dtype=np.str)
    locations = [get_name_with_space(loc) for loc in locations]

    # Create location df (name|longitud|latitud|type)
    locations_coord = []
    geocoder = Nominatim(user_agent = '_')
    for loc in locations:
        location = geocoder.geocode(loc+' Australia')
        if location:
            lon=location.longitude
            lat=location.latitude
            locations_coord.append({"name":loc,"lon":lon,"lat":lat})
        else:
            raise Exception('no coordinates could be found for '+str(loc))

    locations_df = pd.DataFrame.from_dict(locations_coord)
    
    X = locations_df.drop(['name'],axis=1)    
    kmeans = KMeans(n_clusters=6, random_state=0).fit(X)
    locations_df["type"] = kmeans.labels_

    df_['LocationType'] = df_.apply(
        lambda row: int(locations_df.loc[locations_df['name'] == get_name_with_space(row['Location'])].filter(['type']).values[0]), 
        axis=1)

    y = df_.LocationType.values
    onehotencoder = OneHotEncoder(categories='auto',sparse=False)
    y = onehotencoder.fit_transform(y.reshape(-1,1))
    for i in range(y.shape[1]-1):
        df_['LocationType_'+str(i)]=y[:,i]

    return df_

## Feature Selection / New Features

In [5]:
def new_features(df):
    df_ = df.copy()

    # Pressures
    # Mean of colums. When one of the columns is NaN, the mean takes the value of the other column
    df_.Pressure3pm.fillna(df_.Pressure9am, inplace=True)
    df_.Pressure9am.fillna(df_.Pressure3pm, inplace=True)
    press_mean = (df_['Pressure3pm'] + df_['Pressure9am'])/2
    df_['PressureMean'] = press_mean
    df_.drop('Pressure3pm', inplace=True, axis=1)
    df_.drop('Pressure9am', inplace=True, axis=1)

    # Temperatures
    temp_diff = df_['Temp3pm'] - df_['Temp9am']
    df_['TempDiff'] = temp_diff
    # Mean of colums. When one of the columns is NaN, the mean takes the value of the other column
    df_.Temp3pm.fillna(df_.Temp9am, inplace=True)
    df_.Temp9am.fillna(df_.Temp3pm, inplace=True)
    temp_mean = (df_['Temp3pm'] + df_['Temp9am'])/2
    df_['TempMean'] = temp_mean
    df_.drop('Temp3pm', inplace=True, axis=1)
    df_.drop('Temp9am', inplace=True, axis=1)

    # Temperatures max-min
    max_temp_diff = df_['MaxTemp'] - df_['MinTemp']
    df_['TempMaxDiff'] = max_temp_diff
    df_.drop('MinTemp', inplace=True, axis=1)
    df_.drop('MaxTemp', inplace=True, axis=1)

    return df_

## Outliers

In [6]:
def find_skewed_boundaries(df, variable, distance=1.5):
    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)
    lower_boundary = df[variable].quantile(0.25) - (IQR * distance)
    upper_boundary = df[variable].quantile(0.75) + (IQR * distance)
    return upper_boundary, lower_boundary
    
def transform_outliers(df, norm_col, threshold_capped=1.5, threshold_trimmed=1.8,  test_mode=False, limits={},
                        use_manual_limits=False, upper_limit_trim=0, lower_limit_trim=0, upper_limit_cap=0, lower_limit_cap=0, print_output=False):
    
    outliers_limits={}
    # Trimming and capping outliers
    outliers_total = np.array(np.repeat(False,df.shape[0]))
    df_capped = df.copy()
    for col in norm_col:
        
        if not test_mode:
            outliers_limits_col={}
            if use_manual_limits:
                upper_limit, lower_limit = upper_limit_trim, lower_limit_trim
            else:
                upper_limit, lower_limit = find_skewed_boundaries(df, col, threshold_trimmed)

            outliers_limits_col['upper_limit_trim'] = upper_limit
            outliers_limits_col['lower_limit_trim'] = lower_limit
        else:
            upper_limit, lower_limit = limits[col]['upper_limit_trim'], limits[col]['lower_limit_trim']

        outliers = np.where(df[col] > upper_limit, True,
                            np.where(df[col] < lower_limit, True, False))                        
        outliers_total = np.logical_or(outliers_total, outliers)
        
        if print_output:
            print(str(col) + " outliers = "+str(outliers.sum()))
        
        if not test_mode:
            if use_manual_limits:
                upper_limit, lower_limit = upper_limit_cap, lower_limit_cap
            else:
                upper_limit, lower_limit = find_skewed_boundaries(df, col, threshold_capped)

            outliers_limits_col['upper_limit_cap'] = upper_limit
            outliers_limits_col['lower_limit_cap'] = lower_limit
            outliers_limits[col] = outliers_limits_col
        else:
            upper_limit, lower_limit = limits[col]['upper_limit_cap'], limits[col]['lower_limit_cap']

        df_capped[col] = np.where(df[col] > upper_limit, upper_limit,
                            np.where(df[col] < lower_limit, lower_limit, df_capped[col]))

    if print_output:
        print("Total outliers = "+str(outliers_total.sum()))
        
    df_trimmed = df_capped.loc[~(outliers_total)]
    
    if not test_mode:
        return df_trimmed, outliers_limits
    else:
        return df_trimmed

## Simple mean imputation

In [7]:
def mean_imputer(df,cols,test_mode=False,imputer_list=[]):
    df_imputed = df.copy()
    df_imputed['imputed_mean'] = 0
    mean_imputer_list = []

    for k in np.sort(pd.unique(df['LocationType'])):        
        # print("Running mean_imputer for cluster: "+str(k))

        indx=df.loc[df['LocationType'] == k][cols].copy().index # Get index
        
        if not test_mode:
            imputer = SimpleImputer(strategy='mean',add_indicator=True)
        else:
            imputer = imputer_list[k]

        mapper = DataFrameMapper([(cols, imputer)])
        
        # df.loc[df['LocationType'] == k][cols]
        transform_features = mapper.fit_transform(df.loc[df['LocationType'] == k][cols].copy(), 4) # Round 4 digits
        qty_imputed_cols = np.array(transform_features).shape[1]-len(cols) # Number of imputed columns
        imputed_by_mean_imp_col = [str(d) + "_imputed" for d in range(qty_imputed_cols)] # Name of indicators of imputation
        transform_features_df = pd.DataFrame(transform_features, index=indx, columns=cols+imputed_by_mean_imp_col)

        # Add column of imputed indicator. Only one column as an "or" logical operation of all indicators.
        transform_features_df['imputed_mean'] = (transform_features_df[imputed_by_mean_imp_col].sum(axis=1)>0).astype(int)
        df_imputed.loc[indx,cols+['imputed_mean']]=transform_features_df[cols+['imputed_mean']] # Replace in original dataset

        # Check data
        assert(not np.any(df_imputed[df_imputed.LocationType==k][cols].isna().sum()>0))

        mean_imputer_list.append(imputer)

    # Round discrete columns    
    df_imputed.WindDir3pm = df_imputed.WindDir3pm.round()
    
    if not test_mode:
        return df_imputed, mean_imputer_list
    else:
        return df_imputed

## KNN imputation

In [8]:
def knn_imputer(df, cols, neighbors=5,test_mode=False,imputer_list=[]):
    df_imputed = df.copy()
    df_imputed['imputed_knn'] = 0
    knn_imputer_list = []
    
    for k in np.sort(pd.unique(df['LocationType'])):
        # print("Running knn_imputer for cluster: "+str(k))

        indx=df.loc[df['LocationType'] == k][cols].copy().index # Get index
        
        if not test_mode:            
            imputer = KNNImputer(n_neighbors=neighbors,add_indicator=True)
        else:
            imputer = imputer_list[k]            
        mapper = DataFrameMapper([(cols, imputer)])

        # df.loc[df['LocationType'] == k][cols+['LocationType']]
        transform_features = mapper.fit_transform(df.loc[df['LocationType'] == k][cols].copy(), 4) # Round 4 digits
        qty_imputed_cols = np.array(transform_features).shape[1]-len(cols) # Number of imputed columns
        imputed_by_knn_imp_col = [str(d) + "_imputed" for d in range(qty_imputed_cols)] # Name of indicators of imputation
        transform_features_df = pd.DataFrame(transform_features, index=indx, columns=cols+imputed_by_knn_imp_col)
        transform_features_df.isna().sum()

        # Add column of imputed indicator. Only one column as an "or" logical operation of all indicators.
        transform_features_df['imputed_knn'] = (transform_features_df[imputed_by_knn_imp_col].sum(axis=1)>0).astype(int)
        df_imputed.loc[indx,cols+['imputed_knn']]=transform_features_df[cols+['imputed_knn']] # Replace in original dataset

        # Check data
        assert(not np.any(df_imputed[df_imputed.LocationType==k][cols].isna().sum()>0))
        
        knn_imputer_list.append(imputer)

    # Round discrete columns   
    df_imputed.RainToday = df_imputed.RainToday.round()
    df_imputed.WindDir9am = df_imputed.WindDir9am.round()
    df_imputed.WindGustDir = df_imputed.WindGustDir.round()
    if not test_mode:
        return df_imputed, knn_imputer_list
    else:
        return df_imputed

## Cyclic encoding for wind speed

In [9]:
def cyclic_encoding(df, cols, cat):
    '''
    Number of different categories
    '''
    df_ = df.copy()
    for c in cols:
        df_[c+'_cos'] = np.cos(2 * np.pi * (df_[c]/cat))
        df_[c+'_sin'] = np.sin(2 * np.pi * (df_[c]/cat))
    return df_

# Run all

In [10]:
# Load dataset
df = pd.read_csv('weatherAUS.csv')

# Add RainfallTomorrow and RainfallYesterday and delete the first and the last element of each city
df['RainfallTomorrow']=df['Rainfall'].shift(-1)
df['RainfallYesterday']=df['Rainfall'].shift(1)

remove_index = []
for l in pd.unique(df['Location']):
    remove_index.append(df.index[df['Location'] == l][0])
    remove_index.append(df.index[df['Location'] == l][-1])

df.drop(df.index[np.array(remove_index)],inplace=True)

# Drop columns where RainTomorrow is NaN
df = df[df['RainTomorrow'].notna()]

# Split into train/test
X_train_, X_test_, y_train_, y_test_ = train_test_split(
    df.drop(['RainTomorrow'], axis=1),
    df['RainTomorrow'],
    test_size=0.15,
    random_state=0)

# Create df_train and df_test to process all columns together
df_train = X_train_.copy()
df_train['RainTomorrow']=y_train_

df_test = X_test_.copy()
df_test['RainTomorrow']=y_test_

In [12]:
def pre_process_df(df, test_mode=False, param_dict={}):

    if test_mode and not param_dict:
        raise Exception('param_list need for test mode')

    df_ = df.copy()

    # Drop columns
    drop_first_cols=['Evaporation','Sunshine','Cloud9am','Cloud3pm','Date']
    df_ = drop_cols(df_,drop_first_cols)

    # Variable Encoding
    df_ = encode_variables(df_)

    # Create/change some features
    df_ = new_features(df_)

    # Outliers
    norm_col = ['WindGustSpeed','WindSpeed9am','WindSpeed3pm','Humidity9am',
                'Humidity3pm','PressureMean','TempDiff','TempMean','TempMaxDiff']

    norm_rainfall = ['Rainfall','RainfallYesterday','RainfallTomorrow']

    if not test_mode:
        df_, outliers_limits_ = transform_outliers(df_, norm_col)
        df_, outliers_limits_rainfall_ = transform_outliers(df_, norm_rainfall,use_manual_limits=True, 
                                        upper_limit_trim=15, lower_limit_trim=0, upper_limit_cap=10, lower_limit_cap=0)
    else:
        df_ = transform_outliers(df_, norm_col, test_mode=test_mode, limits=param_dict['outliers_limits'])
        df_ = transform_outliers(df_, norm_rainfall, test_mode=test_mode, limits=param_dict['outliers_limits_rainfall'])
    
    # Scaling
    scaled_columns = ['Rainfall','WindGustSpeed','WindSpeed9am','WindSpeed3pm','Humidity9am','Humidity3pm',
                    'RainfallYesterday','RainfallTomorrow','PressureMean','TempDiff','TempMean','TempMaxDiff']

    if not test_mode:
        scaler_ = StandardScaler()
        scaler_.fit(df_[scaled_columns])
    else:
        scaler_ = param_dict['scaler']
        
    df_[scaled_columns] = scaler_.transform(df_[scaled_columns])

    # Imputation
    imputed_by_mean_col = ['Rainfall','WindSpeed9am','WindSpeed3pm','Humidity9am','Humidity3pm',
                        'RainfallYesterday','RainfallTomorrow','TempDiff','TempMean','TempMaxDiff','WindDir3pm']
    imputed_by_knn_col = ['RainToday','WindGustDir','WindGustSpeed','WindDir9am','PressureMean']
    neighbors_col =['Rainfall','WindDir3pm','WindSpeed9am','WindSpeed3pm','Humidity9am','Humidity3pm','TempDiff','TempMean','TempMaxDiff']

    if not test_mode:
        df_, mean_imputer_list_ = mean_imputer(df_,imputed_by_mean_col)
        df_, knn_imputer_list_ = knn_imputer(df_,imputed_by_knn_col+neighbors_col)
    else:
        df_ = mean_imputer(df_, imputed_by_mean_col, test_mode=test_mode, imputer_list=param_dict['mean_imputer_list'])
        df_ = knn_imputer(df_, imputed_by_knn_col+neighbors_col, test_mode=test_mode, imputer_list=param_dict['knn_imputer_list'])

    # Cyclic encoding for wind speed
    wind_dir_colums = ['WindGustDir','WindDir9am','WindDir3pm']
    df_ = cyclic_encoding(df_,cols=wind_dir_colums,cat=16)

    # Normalization
    norm_col_yj = ['Humidity9am','Humidity3pm','PressureMean','TempDiff','TempMean','TempMaxDiff']
    norm_col_qt = ['WindGustSpeed','WindSpeed9am','WindSpeed3pm']
    
    if not test_mode:
        power_yj_ = PowerTransformer(method= 'yeo-johnson')
        qt_transf_ = QuantileTransformer(output_distribution='normal')
        power_yj_.fit(df_[norm_col_yj])
        qt_transf_.fit(df_[norm_col_qt])
    else:
        power_yj_ = param_dict['power_yj']
        qt_transf_ = param_dict['qt_transf']
                
    df_[norm_col_yj] = power_yj_.transform(df_[norm_col_yj])
    df_[norm_col_qt] = qt_transf_.transform(df_[norm_col_qt])

    # Drop columns
    drop_columns = ['Location','LocationType','WindGustDir','WindDir9am','WindDir3pm']
    df_.drop(drop_columns,axis=1,inplace=True)
    if not test_mode:
        param_dict={'outliers_limits': outliers_limits_, 
                'outliers_limits_rainfall': outliers_limits_rainfall_, 
                'scaler': scaler_, 
                'mean_imputer_list': mean_imputer_list_, 
                'knn_imputer_list': knn_imputer_list_, 
                'power_yj': power_yj_, 
                'qt_transf': qt_transf_
        }
        return df_, param_dict

    return df_

In [13]:
df_train_processed, param_dict = pre_process_df(df_train)

In [14]:
df_test_processed = pre_process_df(df_test, test_mode=True, param_dict=param_dict)

### Save data into pickle file

In [15]:
# df_train_processed.to_pickle('./data/df_train_processed.pkl')

# open_file = open('./data/param_dict.pkl', "wb")
# pickle.dump(param_dict, open_file)
# open_file.close()

# df_test_processed.to_pickle('./data/df_test_processed.pkl')

### Load data from pickle file

In [16]:
# df_train_processed = pd.read_pickle('./data/df_train_processed.pkl')

# open_file = open('./data/param_dict.pkl', "rb")
# param_dict = pickle.load(open_file)
# open_file.close()

# df_test_processed = pd.read_pickle('./data/df_test_processed.pkl')

# Models

In [59]:
# X_train = df_train_processed.drop(['RainTomorrow','RainfallTomorrow'],axis=1)
# y_train = df_train_processed.RainTomorrow

# X_test = df_test_processed.drop(['RainTomorrow','RainfallTomorrow'],axis=1)
# y_test = df_test_processed.RainTomorrow

# Split into train/test
X_train, X_cv, y_train, y_cv = train_test_split(
    df_train_processed.drop(['RainTomorrow','RainfallTomorrow'], axis=1),
    df_train_processed['RainTomorrow'],
    test_size=0.2,
    random_state=0)

In [60]:
from sklearn.decomposition import PCA
explained_variance = .95
pca = PCA(n_components=explained_variance).fit(X_train)
X_train_pca = pca.transform(X_train)
X_cv_pca = pca.transform(X_cv)

In [66]:
from sklearn.linear_model import LogisticRegression

# log_reg4=LogisticRegression(C=1.0, class_weight='balanced', penalty='l2',
#                    random_state=None)

logisticRegr = LogisticRegression(class_weight='balanced')
logisticRegr.fit(X_train_pca, y_train)

# Prediction
predictions = logisticRegr.predict(X_cv_pca)

# Use score method to get accuracy of model
score = logisticRegr.score(X_cv_pca, y_cv)
print(score)

0.7516023358495941


In [67]:
from sklearn import metrics

print("precision = "+str(metrics.precision_score(y_cv, predictions)))
print("accuracy = "+str(metrics.accuracy_score(y_cv, predictions)))
print("recall = "+str(metrics.recall_score(y_cv, predictions)))
print("f1_score = "+str(metrics.f1_score(y_cv, predictions)))
print("\nconfusion matrix:")
print(metrics.confusion_matrix(y_cv, predictions))
# print(metrics.classification_report(y_cv, predictions))


precision = 0.38043635312816065
accuracy = 0.7516023358495941
recall = 0.7360916969527537
f1_score = 0.5016193560678224

confusion matrix:
[[13198  4288]
 [  944  2633]]
